In [217]:
import re
import os
import shutil

In [218]:
rule_content = {
    "pcre": "",
    "msg": "",
    "program": "",
    "msg": "",
    "classtype": "",
    "sid": ""
}

In [219]:
def normalize_modsec_conf(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Loại bỏ tất cả comment (dòng bắt đầu bằng # hoặc chỉ chứa # ở đầu)
    lines = [line for line in lines if not line.strip().startswith('#') and line.strip()]

    # Gộp các dòng bị chia thành nhiều dòng
    merged_lines = []
    current_rule = ""
    for line in lines:
        stripped = line.strip()
        if stripped.endswith("\\"):
            current_rule += stripped[:-1] + " "  # Loại bỏ dấu '\' và thêm khoảng trắng
        else:
            current_rule += stripped
            merged_lines.append(current_rule)
            current_rule = ""

    with open(output_file, 'w', encoding='utf-8') as f:
        for line in merged_lines:
            f.write(line + "\n")

# Sử dụng hàm
# normalize_modsec_conf(input_path, output_path)

In [220]:
def generate_lt_regex(value):
    digits = list(str(value))  # Split value into individual digits
    # print(digits)
    n = len(digits)  # Length of the number
    # Khởi tạo regex
    regex_parts = []

    if n == 1:
        return f"^[0-{int(digits[0])-1}]$"

    # Trường hợp 1: Số có ít chữ số hơn n
    if n > 1:
        regex_parts.append(f"^[0-9]{{1,{n-1}}}$")

    # Trường hợp 2: Số có đúng n chữ số nhưng nhỏ hơn value
    for i in range(n):
        if int(digits[i]) > 0:
            prefix = "".join(digits[:i])
            current_digit = int(digits[i])
            if current_digit > 0:
                if i == 0:
                    if current_digit > 1:
                        regex_parts.append(f"^[{1 if n > 1 else 0}-{current_digit-1}][0-9]{{{n-1}}}$")
                    elif current_digit == 1 and n > 1:
                        regex_parts.append(f"^[0-9]{{{n-1}}}$")
                else:
                    regex_parts.append(f"^{prefix}[0-{current_digit-1}][0-9]{{{n-1-i}}}$")
    regex = "|".join(regex_parts)

    return regex if regex else "^$"


print(generate_lt_regex(123))
print(generate_lt_regex(1))
def generate_eq_regex(value, is_not=False):
    """Tạo regex cho phép toán = value"""
    return f"(^0*{value}$)" if not is_not else f"(^(?!0*{value}$).+)"

def generate_le_regex(value):
    """Tạo regex cho phép toán ≤ value"""
    return generate_lt_regex(value + 1)

print(generate_le_regex(123))

def generate_ge_regex(value):
    """Tạo regex cho phép toán ≥ value (phủ định của < value)"""
    return f"^(?!({generate_lt_regex(value)})).*"
print(generate_ge_regex(123))
def generate_gt_regex(value):
    """Tạo regex cho phép toán > value (phủ định của ≤ value)"""
    leng=len(str(value))
    return f"(^(?!({generate_le_regex(value)})).*)" +f"|(^0*"+ f"[0-9]{{{leng+1},}}$)"
print(generate_gt_regex(123))


^[0-9]{1,2}$|^[0-9]{2}$|^1[0-1][0-9]{1}$|^12[0-2][0-9]{0}$
^[0-0]$
^[0-9]{1,2}$|^[0-9]{2}$|^1[0-1][0-9]{1}$|^12[0-3][0-9]{0}$
^(?!(^[0-9]{1,2}$|^[0-9]{2}$|^1[0-1][0-9]{1}$|^12[0-2][0-9]{0}$)).*
(^(?!(^[0-9]{1,2}$|^[0-9]{2}$|^1[0-1][0-9]{1}$|^12[0-3][0-9]{0}$)).*)|(^0*[0-9]{4,}$)


In [221]:
def convert_to_prce(component, is_not=False):
    elements = component.split(" ")
    operator = elements[0]

    # Hàm chuyển đổi giá trị thành số nguyên (nếu không được, trả về 0)
    def to_int(value):
        try:
            return int(value)
        except ValueError:
            return 0
    '''
    test:
    GET  hello world nGET
    GETn  hello world GET
    hello GET
    hello nGET
    hello .GET
    hello GET.
    '''
    match operator:

        case _ if operator.startswith("detect"):
            return None
        case "rx":
            return " ".join(elements[1:])
            # return component
        case "beginsWith":
            return f"^{'(?!' if is_not else ''}{elements[1]}{')' if is_not else ''}.*"
        case "endsWith":
            return f"^(?!.*{elements[1]}$).+" if is_not else f".*{elements[1]}$"
        case "contains":
            return f"^(?!.*{elements[1]}).+" if is_not else f".*{elements[1]}.*"
        case "containsWord":
            return f"^(?!.*\\b{elements[1]}\\b).+" if is_not else f"\\b{elements[1]}\\b"
        case "eq":
            value = to_int(elements[1])
            # return f"^({value})$" if not is_not else f"^(?!({value})$).+"
            return generate_eq_regex(value)
        case "ge":
            value = to_int(elements[1])
            return generate_ge_regex(value)
        case "gt":
            value = to_int(elements[1])
            return generate_gt_regex(value)
        case "le":
            value = to_int(elements[1])
            return generate_le_regex(value)
        case "lt":
            value = to_int(elements[1])
            return generate_lt_regex(value)

In [222]:
def convert_rule(rule):

    # keyword list content
    # sid = re.search(r"id:(\d+),", rule).group(1) if re.search(r"id:(\d+),", rule).group(1) else None
    #
    # msg = re.search(r"msg:'([^']+)'", rule).group(1) if re.search(r"msg:'([^']+)'", rule) else None
    sid_match = re.search(r"id:(\d+),", rule)
    sid = sid_match.group(1) if sid_match else None

    msg_match = re.search(r"msg:'([^']+)'", rule)
    msg = msg_match.group(1) if msg_match else None

    operator = re.search(r"@([^\"]+)\"", rule)
    if len(operator.group(1).split(" ")) > 1:
        pcre = convert_to_prce(operator.group(1))
        program = ""
        classtype = ""
        # write rule sagan
        sagan_rule= f"alert any any any -> any any (msg:\"{msg}\"; pcre:\"{pcre}\"; normalize; program:{program}; threshold:type limit,track by_src,count 1,seconds 60; classtype:{classtype}; sid:{sid}; rev:1;)"


        return sagan_rule
    else:
        return None



In [223]:
def convert_file(input_file_path, output_file_path):
    # read input file
    with open(input_file_path, 'r', encoding="utf-8") as file:
        lines = file.readlines()

    # process each rule
    for line in lines:
        if line.startswith("SecRule"):
            sagan_rule = convert_rule(line)

            # record the results
            with open(output_file_path, 'a', encoding="utf-8") as file:
                file.write(f"{sagan_rule}\n")

    print(f"Processed file: {input_file_path} -> Save to file: {output_file_path}")

def convert_mod_to_sagan(modsecurity_path,standardized_path, sagan_path):
    # create sagan rules folder
    if os.path.exists(standardized_path):
        shutil.rmtree(standardized_path)
    os.makedirs(standardized_path)

    if os.path.exists(sagan_path):
        shutil.rmtree(sagan_path)
    os.makedirs(sagan_path)

    for file in os.listdir(modsecurity_path):
        input_file_path = os.path.join(modsecurity_path, file)
        output_file_path = os.path.join(standardized_path, file)
        normalize_modsec_conf(input_file_path, output_file_path)

    # process each file
    for filename in os.listdir(standardized_path):
        # get input file and create output file.
        input_file_path = os.path.join(standardized_path, filename)
        file_name_without_extension = os.path.splitext(os.path.basename(filename))[0]
        output_file_path = os.path.join(sagan_path, f"sagan_{file_name_without_extension}.rules")

        # convert each file
        convert_file(input_file_path, output_file_path)

    print("Completed processing all files!")


In [224]:
def main():
    path_data = "data"
    sagan_path = "result"
    standardized_path = "standardized"
    convert_mod_to_sagan(path_data,standardized_path,sagan_path)

if __name__ == "__main__":
    main()

Processed file: standardized\REQUEST-942-APPLICATION-ATTACK-SQLI.conf -> Save to file: result\sagan_REQUEST-942-APPLICATION-ATTACK-SQLI.rules
Completed processing all files!
